### Casper's Ghost Kitchen Initializer

Select `Run All` to create the job `Casper's Initializer`. Once this notebook has been run you can initialize the Casper's environment using the job, which also allows you to selectively create only parts of the demo (broken into tasks within the job).

**After running this notebook:**
- Navigate to Jobs & Pipelines in the left sidebar of your Databricks workspace
- Find and run the `Casper's Initializer` job
- You can pick a subset of tasks to run if you want. The `Raw_Data` and `Lakeflow_Declarative_Pipeline` tasks are required, but downstream tasks are demo-specific and you can run whichever ones you need.

**Important Note on Catalog Name:**
The default catalog name is `caspers`. If you're working in a metastore that spans multiple workspaces and another workspace has already used this catalog name, you'll need to specify a different name using the `CATALOG` parameter. Catalog names must be unique within a metastore.

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
CATALOG = dbutils.widgets.get("CATALOG")

In [ ]:
from databricks.sdk.service.jobs import JobSettings as Job
import os

Casper_s_Initializer = Job.from_dict(
    {
        "name": "Casper's Initializer",
        "tasks": [
            {
                "task_key": "Raw_Data",
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/raw_data"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Lakeflow_Declarative_Pipeline",
                "depends_on": [
                    {
                        "task_key": "Raw_Data",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/lakeflow"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Refund_Recommender_Agent",
                "depends_on": [
                    {
                        "task_key": "Lakeflow_Declarative_Pipeline",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/refunder_agent"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Refund_Recommender_Stream",
                "depends_on": [
                    {
                        "task_key": "Lakeflow_Declarative_Pipeline",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/refunder_stream"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Complaint_Agent",
                "depends_on": [
                    {
                        "task_key": "Lakeflow_Declarative_Pipeline",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/complaint_agent"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Complaint_Generator_Stream",
                "depends_on": [
                    {
                        "task_key": "Lakeflow_Declarative_Pipeline",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/complaint_generator_stream"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Complaint_Agent_Stream",
                "depends_on": [
                    {
                        "task_key": "Complaint_Agent",
                    },
                    {
                        "task_key": "Complaint_Generator_Stream",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/complaint_agent_stream"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Complaint_Lakebase",
                "depends_on": [
                    {
                        "task_key": "Complaint_Agent_Stream",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/complaint_lakebase"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Lakebase_Reverse_ETL",
                "depends_on": [
                    {
                        "task_key": "Refund_Recommender_Stream",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/lakebase"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Databricks_App_Refund_Manager",
                "depends_on": [
                    {
                        "task_key": "Lakebase_Reverse_ETL",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./stages/apps"),
                    "source": "WORKSPACE",
                },
            },
        ],
        "queue": {
            "enabled": True,
        },
        "parameters": [
            {
                "name": "CATALOG",
                "default": f"{CATALOG}",
            },
            {
                "name": "EVENTS_VOLUME",
                "default": "events",
            },
            {
                "name": "LLM_MODEL",
                "default": "databricks-meta-llama-3-3-70b-instruct",
            },
            {
                "name": "REFUND_AGENT_ENDPOINT_NAME",
                "default": "caspers_refund_agent",
            },
            {
                "name": "COMPLAINT_AGENT_ENDPOINT_NAME",
                "default": "caspers_complaint_agent",
            },
            {
                "name": "COMPLAINT_RATE",
                "default": "0.15",
            },
            {
                "name": "SIMULATOR_SCHEMA",
                "default": "simulator",
            },
            {
                "name": "LOCATIONS",
                "default": "sanfrancisco.json",
            },
        ],
        "performance_target": "PERFORMANCE_OPTIMIZED",
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
w.jobs.create(**Casper_s_Initializer.as_shallow_dict())

In [ ]:
print(f"✅ Job created! Go to Jobs & Pipelines → 'Casper's Initializer' to run it")